In [8]:
%load_ext autoreload
%autoreload 2

from utils import *
from compare import compare



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
val_labels = loadLabels("data/val_labels/", 100)
print("labels of {} images loaded".format(str(len(val_labels))))
print('first data:')
print(val_labels[0])

labels of 100 images loaded
first data:
[(153, 125), (79, 151), (100, 153), (273, 177), (262, 202), (204, 206), (313, 228), (273, 230), (66, 255), (178, 285), (143, 298), (39, 304), (305, 312), (32, 334), (61, 347), (80, 349), (605, 44), (441, 94), (448, 109), (445, 117), (429, 165), (426, 173), (538, 177), (519, 209), (674, 212), (426, 223), (668, 227), (599, 266), (596, 275), (453, 293), (801, 98), (791, 137), (1058, 153), (1047, 201), (1049, 228), (817, 234), (1157, 269), (907, 282), (810, 285), (1125, 318), (898, 329), (1051, 333), (802, 341), (1086, 365), (1214, 103), (1328, 144), (1184, 147), (1195, 161), (1373, 194), (1214, 204), (1413, 240), (1487, 283), (1193, 292), (1180, 301), (1238, 366)]


In [15]:
resultPath = "data/val_results/"
archs = ["newpeaksv2", "newpeaksv3","newpeaksv4", "newpeaksv5", "yolov3-peaks"]

for arch in archs:
    print("******************************************")
    print("*****       testing " + arch)
    print("******************************************")
    val_results = loadResults(resultPath+arch+".json")
    print("results of {} images loaded".format(str(len(val_results))))
    print('first data has {} objects'.format(str(len(val_results[0]))))
    
    val_comp_results, val_comp_stats = compare( val_labels, val_results, 0.1, False )
    val_sen, val_pre, val_iou, val_dist = val_comp_stats
    print('val sensitivity:', val_sen)
    print('val precision:', val_pre)
    print('val avg. IOU:', val_iou)
    print('val avg. dist.:', val_dist)

******************************************
*****       testing newpeaksv2
******************************************
data/val_results/newpeaksv2.json
results of 1000 images loaded
first data has 708 objects
val sensitivity: 0.7374355670103093
val precision: 0.5296159185562239
val avg. IOU: 0.2642410474936736
val avg. dist.: 2.1425188417999963
******************************************
*****       testing newpeaksv3
******************************************
data/val_results/newpeaksv3.json
results of 1000 images loaded
first data has 326 objects
val sensitivity: 0.579735824742268
val precision: 0.48276324614352784
val avg. IOU: 0.23462614910151283
val avg. dist.: 2.4428146638933295
******************************************
*****       testing newpeaksv4
******************************************
data/val_results/newpeaksv4.json
results of 1000 images loaded
first data has 133 objects
val sensitivity: 0.3587306701030928
val precision: 0.6559646539027982
val avg. IOU: 0.2011805772239569